In [ ]:
# SPDX-License-Identifier: GPL-3.0
# Copyright (c) 2024,2025 Ryusei Tanaka.
# All Rights Reserved.
# email : s24g354@kagawa-u.ac.jp


# ファイルパス入力↓

In [ ]:
log_dirs = []

# パケットロス率計算

In [ ]:
import os
import glob

def read_files(recv_log_data, send_log_data):
    total_recv_data = 0  # 受信データの合計値
    with open(recv_log_data, "r", encoding="utf-8") as f1:
        lines = f1.readlines()
        if lines:
            chunk_size = int(lines[0].split(",")[2])  # 最初の行のチャンクサイズを取得
            total_recv_data = (len(lines)) * chunk_size  # 最初の行を除いた行数 × チャンクサイズ
    # 送信データの合計値を行数×チャンクサイズで計算
    total_send_data = 0
    with open(send_log_data, "r", encoding="utf-8") as f2:
        lines = f2.readlines()
        if lines:
            chunk_size = int(lines[0].split(",")[1])  # 最初の行のチャンクサイズを取得
            total_send_data = (len(lines)) * chunk_size  # 最初の行を除いた行数 × チャンクサイズ
    
    return total_recv_data, total_send_data

# 送信ログとGPSログのあるフォルダを複数指

for log_dir in log_dirs:
    send_files = sorted(glob.glob(os.path.join(log_dir, "tcp_send_data_*.txt")))
    recv_files = sorted(glob.glob(os.path.join(log_dir, "serial_recv_data_*.txt")))
    
    for send_file, recv_file in zip(send_files, recv_files):
        all_recv_log_data, all_send_log_data = read_files(recv_file, send_file)
        
        print(f"Folder: {log_dir}")
        print(f"Recv File: {recv_file}")
        print(f"Send File: {send_file}")
        print("File 2 (Total send data):", all_send_log_data)
        print("File 1 (All recv data):", all_recv_log_data)
        print("Diff data:", all_send_log_data - all_recv_log_data)
        print("Diff data rate:",100- (all_recv_log_data / all_send_log_data * 100) if all_send_log_data > 0 else 0, "%")
        print("-" * 50)


# スループット計算(受信ログ)

In [ ]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt

# 送信ログとGPSログのあるフォルダを複数指定
transmit_files = []
gps_files = []
recv_files = []

for log_dir in log_dirs:
    transmit_files.extend(sorted(glob.glob(os.path.join(log_dir, "tcp_send_data_*.txt"))))
    gps_files.extend(sorted(glob.glob(os.path.join(log_dir, "Recv_GPS_Data_*.txt"))))
    recv_files.extend(sorted(glob.glob(os.path.join(log_dir, "serial_recv_data_*.txt"))))

# 各受信ログのスループットを計算し、グラフを描画
plt.figure(figsize=(10, 6))

for file_name in recv_files:
    try:
        # ファイルを読み込む
        data = pd.read_csv(
            file_name,  
            header=None, 
            names=["timestamp", "sequence", "size"],  
            sep=",",  
            engine='python'
        )

        # タイムスタンプをdatetime形式に変換
        data["timestamp"] = pd.to_datetime(data["timestamp"])

        # 経過時間を計算
        start_time = data["timestamp"].iloc[0]
        data["elapsed_time"] = (data["timestamp"] - start_time).dt.total_seconds()
        data["second"] = data["elapsed_time"].astype(int)  # 経過秒数を整数に

        # 1秒ごとのスループットを計算 (Byte/sec → Kbps)
        throughput_per_sec = data.groupby("second")["size"].sum() * 8 / 1024

        # 瞬間最高スループットを取得
        peak_throughput = throughput_per_sec.max()
        peak_time = throughput_per_sec.idxmax()

        # スループットをプロット
        plt.plot(throughput_per_sec.index, throughput_per_sec, linestyle='-', marker='o', label=os.path.basename(file_name))
        plt.scatter(peak_time, peak_throughput, color='r', s=100, zorder=3, label=f"Peak {os.path.basename(file_name)}")

        # 結果を表示
        print(f"ファイル名: {file_name}")
        print(f"最高スループット: {peak_throughput:.2f} Kbps")
        print(f"最高スループット発生時刻 (経過秒数): {peak_time} 秒")
        print("-" * 40)

    except Exception as e:
        print(f"ファイル {file_name} の処理中にエラーが発生しました: {e}")

# グラフの設定
plt.xlabel("Elapsed Time (seconds)")
plt.ylabel("Throughput (Kbps)")
plt.title("Throughput over Elapsed Time")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


# 遅延計算　縦軸遅延時間、横軸経過時間

In [ ]:
import matplotlib.pyplot as plt
import os
import statistics
from datetime import datetime

def read_log_file(file_path):
    """ログファイルを読み込んでリストとして返す"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines() if line.strip()]

def extract_time_and_sequence_send(log_line):
    """送信ログの時間とシーケンス番号を抽出"""
    parts = log_line.split(",")
    time = datetime.strptime(parts[0], "%Y-%m-%d %H:%M:%S.%f")
    seq_num = int(parts[2])
    return time, seq_num

def extract_time_and_sequence_recv(log_line):
    """受信ログの時間とシーケンス番号を抽出"""
    parts = log_line.split(",")
    time = datetime.strptime(parts[0], "%Y-%m-%d %H:%M:%S.%f")
    seq_num = int(parts[1])
    return time, seq_num

def find_log_files(folder_path):
    """指定フォルダ内の送信ログと受信ログを検索"""
    send_log = None
    recv_log = None
    
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if "tcp_send_data" in file:
            send_log = file_path
        elif "serial_recv_data" in file:
            recv_log = file_path
    
    return send_log, recv_log

def calculate_time_differences(send_path, recv_path):
    """対応するシーケンス番号同士の時間差を計算"""
    send_log = read_log_file(send_path)
    recv_log = read_log_file(recv_path)
    
    send_times = {}
    for line in send_log:
        time, seq = extract_time_and_sequence_send(line)
        send_times[seq] = time  # 最新の送信時間を記録（上書き）

    time_diffs = []
    seq_diffs = []  # シーケンス番号を保持
    for line in recv_log:
        time, seq = extract_time_and_sequence_recv(line)
        if seq in send_times:
            time_diff = (time - send_times[seq]).total_seconds()
            time_diffs.append(time_diff)
            seq_diffs.append(seq)  # 対応するシーケンス番号を追加
    
    return time_diffs, seq_diffs

def plot_time_differences(time_diffs, min_diff, max_diff, mode_diff, folder_name):
    """時間差のリストをプロット (縦軸を対数軸に設定)"""
    plt.figure(figsize=(10, 5))
    plt.plot(time_diffs, marker='o', linestyle='-', color='b', label='Time Difference (s)')
    
    if mode_diff is not None:
        plt.axhline(y=mode_diff, color='r', linestyle='--', label=f'Mode: {mode_diff:.6f} s')
    
    plt.axhline(y=min_diff, color='g', linestyle=':', label=f'Min: {min_diff:.6f} s')
    plt.axhline(y=max_diff, color='m', linestyle=':', label=f'Max: {max_diff:.6f} s')

    plt.yscale('log')  # 縦軸を対数スケールに設定
    plt.xlabel('Packet sequence number')
    plt.ylabel('Time Difference (seconds) (log scale)')
    plt.title(f'Time Differences in {folder_name}')
    plt.legend()
    plt.grid()
    plt.show()

def process_folder(folder_path):
    send_path, recv_path = find_log_files(folder_path)
    if not send_path or not recv_path:
        print(f"[{folder_path}] 送信ログまたは受信ログが見つかりません。")
        return
    
    differences, sequences = calculate_time_differences(send_path, recv_path)

    if differences:
        min_diff = min(differences)
        max_diff = max(differences)
        
        min_seq = sequences[differences.index(min_diff)]
        max_seq = sequences[differences.index(max_diff)]

        try:
            mode_diff = statistics.mode(differences)
            print(f"[{folder_path}] 最小: {min_diff:.6f} 秒 (シーケンス: {min_seq}), 最大: {max_diff:.6f} 秒 (シーケンス: {max_seq}), 最頻値: {mode_diff:.6f} 秒")
            plot_time_differences(differences, min_diff, max_diff, mode_diff, folder_path)
        except statistics.StatisticsError:
            print(f"[{folder_path}] 最小: {min_diff:.6f} 秒 (シーケンス: {min_seq}), 最大: {max_diff:.6f} 秒 (シーケンス: {max_seq}), 最頻値が一意に定まりません。")
            plot_time_differences(differences, min_diff, max_diff, None, folder_path)

if __name__ == "__main__":

    for log_dir in log_dirs:
        process_folder(log_dir)


# # 距離plotコード 縦軸遅延時間(対数軸)、横軸移動距離

In [ ]:
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib # 日本語化matplotlib
import matplotlib.patches as mpatches
import glob
import os
import re
from geopy.distance import geodesic
from datetime import datetime

# 基準地点（北緯3417'33", 東経134°3'52"）
ref_point = (34 + 17/60 + 33/3600, 134 + 3/60 + 52/3600, 35)  # (緯度, 経度, 高さ)

def parse_gpgga(nmea_sentence):
    """NMEAのGPGGA文から緯度・経度・高度を抽出"""
    parts = nmea_sentence.split(',')
    if len(parts) < 10:
        return None
    lat = float(parts[2][:2]) + float(parts[2][2:]) / 60
    lon = float(parts[4][:3]) + float(parts[4][3:]) / 60
    if parts[3] == 'S': lat = -lat
    if parts[5] == 'W': lon = -lon
    altitude = float(parts[9])  # 高度 (メートル)
    return (lat, lon, altitude)

def read_gps_log(file_path):
    """GPSログを読み込む"""
    gps_data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or "Satellite Status:" in line:
                continue
            try:
                if "$GPGGA" not in line:
                    continue
                parts = line.split(" ", 3)
                if len(parts) < 4:
                    continue
                time_str = parts[0] + " " + parts[1]
                nmea_sentence = parts[3]
                gps_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
                parsed_data = parse_gpgga(nmea_sentence)
                if parsed_data:
                    gps_data.append((gps_time, parsed_data))
            except ValueError:
                continue
    return gps_data

def find_log_files(folder_path):
    """指定フォルダ内の送信ログ、受信ログ、GPSログを検索"""
    send_log, recv_log, gps_log = None, None, None
    
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if "tcp_send_data" in file:
            send_log = file_path
        elif "serial_recv_data" in file:
            recv_log = file_path
        elif "Recv_GPS_Data" in file:
            gps_log = file_path
    
    return send_log, recv_log, gps_log

def read_log_file(file_path):
    """ログファイルを読み込んでリストとして返す"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines() if line.strip()]

def extract_time_and_sequence_send(log_line):
    """送信ログの時間とシーケンス番号を抽出"""
    parts = log_line.split(",")
    time = datetime.strptime(parts[0], "%Y-%m-%d %H:%M:%S.%f")
    seq_num = int(parts[1]) // 1024
    return time, seq_num

def extract_time_and_sequence_recv(log_line):
    """受信ログの時間とシーケンス番号を抽出"""
    parts = log_line.split(",")
    time = datetime.strptime(parts[0], "%Y-%m-%d %H:%M:%S.%f")
    seq_num = int(parts[1])
    return time, seq_num

def calculate_time_differences(send_path, recv_path):
    """対応するシーケンス番号同士の時間差を計算"""
    send_log = read_log_file(send_path)
    recv_log = read_log_file(recv_path)
    
    send_entries = [extract_time_and_sequence_send(line) for line in send_log]  # (time, seq)
    recv_entries = [extract_time_and_sequence_recv(line) for line in recv_log]  # (time, seq)

    send_idx = 0
    recv_idx = 0
    time_diffs = []

    while send_idx < len(send_entries) and recv_idx < len(recv_entries):
        send_time, send_seq = send_entries[send_idx]
        recv_time, recv_seq = recv_entries[recv_idx]

        if send_seq == recv_seq:
            time_diff = (recv_time - send_time).total_seconds()
            time_diffs.append((recv_time, time_diff, send_seq))
            # print(f"Seq: {send_seq}, Send Time: {send_time}, Recv Time: {recv_time}, Time Diff: {time_diff} sec")
            send_idx += 1
            recv_idx += 1
        elif send_seq < recv_seq:
            send_idx += 1
        else:
            recv_idx += 1

    return time_diffs

def process_logs(folder):
    """送信・受信・GPSログを処理し、データを返す"""
    send_file, recv_file, gps_file = find_log_files(folder)
    if not send_file or not recv_file or not gps_file:
        print(f"Skipping {folder} due to missing data.")
        return [], [], []
    
    time_diffs  = calculate_time_differences(send_file, recv_file)
    gps_data = read_gps_log(gps_file)
    if not time_diffs or not gps_data:
        return [], [], []
    
    seq_seen = Counter()  # 各シーケンス番号の出現回数を記録

    distances = []
    delay_times = []
    colors = []

    for time, time_diff, send_seq in time_diffs:
        nearest_gps = min(gps_data, key=lambda x: abs((time - x[0]).total_seconds()))
        gps_point = nearest_gps[1]
        distance = geodesic((gps_point[0], gps_point[1]), (ref_point[0], ref_point[1])).meters
        distances.append(distance)
        delay_times.append(time_diff)
        
        # 何回目の出現かを記録
        seq_seen[send_seq] += 1
        count = seq_seen[send_seq]

        # 色の設定
        if count == 2:
            colors.append('yellow')    # 2回目: 赤
        elif count == 3:
            colors.append('red') # 3回目: 黄
        elif count >= 4:
            colors.append('black')  # 4回目以上: 黒
        else:
            colors.append('blue')   # 初回: 青

    return distances, delay_times, colors


plt.figure(figsize=(10, 6))

# マーカーの種類
marker_styles = ["s", "^", "o"]  
colors_map = {
    "first": "blue",
    "retry1": "yellow",
    "retry2": "red",
    "retry3+": "black"
}

# 各フォルダの処理
for i, log_dir in enumerate(log_dirs):
    distances, delay_times, point_colors = process_logs(log_dir)
    if not distances or not delay_times:
        continue
    
    categorized_data = {"first": [], "retry1": [], "retry2": [], "retry3+": []}
    
    # データの分類
    for d, t, c in zip(distances, delay_times, point_colors):
        if c == "blue":
            categorized_data["first"].append((d, t))
        elif c == "yellow":
            categorized_data["retry1"].append((d, t))
        elif c == "red":
            categorized_data["retry2"].append((d, t))
        else:
            categorized_data["retry3+"].append((d, t))
    
    # 各カテゴリーごとにプロット
    for category, data in categorized_data.items():
        if data:
            d_vals, t_vals = zip(*data)
            label = f"測定{i+1}" if category == "first" else f"測定{i+1} ({category.replace('retry', '')}回再送)"
            plt.scatter(d_vals, t_vals, color=colors_map[category], label=label, marker=marker_styles[i], facecolors='none')

plt.xlim(0, 1200)
plt.ylim(10**-1, 10**2)

# 0 を含む対数スケール (symlog) に設定
plt.yscale('log')

# # 縦軸の目盛りを明示的に指定
yticks = [10**-1, 10**0, 10**1, 10**2]
plt.yticks(yticks, labels=[r"$10^{-1}$", r"$10^{0}$", r"$10^{1}$", r"$10^{2}$"] , minor=False)

plt.grid(False)
# 軸ラベルのフォントサイズを大きくする
plt.xlabel("通信距離 (m)", fontsize=24)
plt.ylabel("送信遅延時間 (秒)", fontsize=24)

# 軸目盛のフォントサイズを大きくする
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

# 凡例のフォントサイズを大きくする
plt.legend(fontsize=14)

plt.grid()
plt.show()

# 距離plotコード

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import re
from geopy.distance import geodesic
from datetime import datetime

# 基準地点（北緯3417'33", 東経134°3'52"）
ref_point = (34 + 17/60 + 33/3600, 134 + 3/60 + 52/3600, 35)  # (緯度, 経度, 高さ)

def parse_gpgga(nmea_sentence):
    """NMEAのGPGGA文から緯度・経度・高度を抽出"""
    parts = nmea_sentence.split(',')
    if len(parts) < 10:
        return None  # フォーマット異常チェック
    lat = float(parts[2][:2]) + float(parts[2][2:]) / 60
    lon = float(parts[4][:3]) + float(parts[4][3:]) / 60
    if parts[3] == 'S': lat = -lat
    if parts[5] == 'W': lon = -lon
    altitude = float(parts[9])  # 高度 (メートル)
    return (lat, lon, altitude)

def read_transmit_log(file_path):
    """送信ログを読み込む"""
    transmit_data = []
    with open(file_path, "r") as f:
        for line in f:
            if not line.strip():
                continue
            try:
                time_str, size , seq= line.strip().split(", ")
                transmit_data.append((datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S.%f"), int(size)))
            except ValueError:
                continue  # 予期しないフォーマットの行をスキップ
    return transmit_data

def read_gps_log(file_path):
    """GPSログを読み込む"""
    gps_data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or "Satellite Status:" in line:
                continue
            try:
                if "$GPGGA" not in line:
                    continue
                parts = line.split(" ", 3)
                if len(parts) < 4:
                    continue
                time_str = parts[0] + " " + parts[1]
                nmea_sentence = parts[3]
                gps_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
                parsed_data = parse_gpgga(nmea_sentence)
                if parsed_data:
                    gps_data.append((gps_time, parsed_data))
            except ValueError:
                continue
    return gps_data

def get_chunk_size_from_folder(log_dir):
    """フォルダ名からチャンクサイズを取得する"""
    match = re.search(r'_(\d{2,})', log_dir)
    return  1024  # デフォルト値を1024に設定

def process_logs(transmit_file, gps_file, chunk_size):
    """送信ログとGPSログを処理し、データを返す"""
    transmit_data = read_transmit_log(transmit_file)
    gps_data = read_gps_log(gps_file)
    if not transmit_data or not gps_data:
        print(f"Skipping {transmit_file} and {gps_file} due to missing data.")
        return [], []
    
    # GPSデータの補間
    gps_df = pd.DataFrame(gps_data, columns=["time", "gps"])
    gps_df.set_index("time", inplace=True)
    gps_df["lat"] = gps_df["gps"].apply(lambda x: x[0])
    gps_df["lon"] = gps_df["gps"].apply(lambda x: x[1])
    gps_df["altitude"] = gps_df["gps"].apply(lambda x: x[2])
    gps_df.drop(columns=["gps"], inplace=True)
    gps_df = gps_df[~gps_df.index.duplicated(keep="first")]
    gps_df = gps_df.resample("1s").interpolate()
    gps_data = list(zip(gps_df.index, zip(gps_df["lat"], gps_df["lon"], gps_df["altitude"])))
    
    distances = []
    data_sizes = []
    for send_time, size in transmit_data:
        if not gps_data:
            continue
        nearest_gps = min(gps_data, key=lambda x: abs((send_time - x[0]).total_seconds()))
        gps_point = nearest_gps[1]
        distance = geodesic((gps_point[0], gps_point[1]), (ref_point[0], ref_point[1])).meters
        distances.append(distance)
        data_sizes.append(size / chunk_size)
    
    return distances, data_sizes

# 送信ログとGPSログのあるフォルダを複数指定
log_dirs = ["send_log/300kbps_16_1" , "send_log/300kbps_16_2"]
transmit_files = []
gps_files = []
chunk_sizes = []
for log_dir in log_dirs:
    transmit_files.extend(sorted(glob.glob(os.path.join(log_dir, "tcp_send_data_*.txt"))))
    gps_files.extend(sorted(glob.glob(os.path.join(log_dir, "Recv_GPS_Data_*.txt"))))
    chunk_sizes.append(get_chunk_size_from_folder(log_dir))

# 色のリスト
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
plt.figure(figsize=(10, 6))

# ファイルごとにプロット
for i, (t_file, g_file, chunk_size) in enumerate(zip(transmit_files, gps_files, chunk_sizes)):
    distances, data_sizes = process_logs(t_file, g_file, chunk_size)
    if distances and data_sizes:
        plt.plot(distances, data_sizes, marker='o', linestyle='-', color=colors[i % len(colors)], label=os.path.basename(t_file))

plt.xlabel("Distance from Reference Point (meters)")
plt.ylabel("Transmitted sequence number")
plt.title("Data Transmission vs. Distance for Multiple Logs")
plt.legend()
plt.grid()
plt.show()
